In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import ase
import numpy as np
import ase.io as aio

from toolz.curried import pipe, curry, compose

In [3]:
import filters
import stats as astat
import utilities as utils

In [4]:
import matplotlib.pyplot as plt

import matplotlib as mpl
mpl.rcParams['lines.linewidth'] = 2
mpl.rcParams['lines.color'] = 'r'
plt.style.use('ggplot')
mpl.rcParams['figure.figsize'] = [12.0, 9.0]
mpl.rcParams['figure.dpi'] = 80
mpl.rcParams['savefig.dpi'] = 100

mpl.rcParams['font.size'] = 14
mpl.rcParams['legend.fontsize'] = 'large'
mpl.rcParams['figure.titlesize'] = 'medium'

In [5]:
def draw(im):
    im1 = np.squeeze(im)
    plt.imshow(im1)
    plt.colorbar()
    plt.grid(False)
    plt.show()

In [6]:
from ase import Atoms
get_atoms = lambda a: Atoms(["Cu"]*len(a),
                            positions=a.get_positions(),
                            cell=a.get_cell(), 
                            pbc=[1, 1, 1], 
                            tags=a.get_tags())

runs = np.loadtxt("runs.txt").astype(int)

## Estimate Distrubance in 3D

In our use of both two point statistics and rdf, we first extract atoms that constitute the substructure of interest (e.g. cascade or grain boundary) - then utilize either of these two metrics to aggregate the local environment around all the atoms constituting the substructure.

* Load coordinates of all atoms in 'coords_all'
* Load coordinates of disturbed atom in 'coords_sub' 
    * (They can be computed using ovito - Common Neighbour Analysis and saved in an xyz file )
* The xyz file if available can be read with ase usin the following code snippet:

         atom = aio.read("file_name.xyz")
         coords = atom.get_positions()

### When computing for damaged structures, atoms corresponding to the damaged region are considered.

In [8]:
# from ase.build import bulk
# atoms = bulk('Cu', 'fcc', a=3.6, cubic=True).repeat([10,10,10])

In [12]:
# @curry
# def get_scaled_positions(coords, cell, pbc, wrap=True):
#     """Get positions relative to unit cell i.e. fractional coordinates.
#     If wrap is True, atoms outside the unit cell will be wrapped into
#     the cell in those directions with periodic boundary conditions
#     so that the scaled coordinates are between zero and one.
#     """
#     fractional = np.linalg.solve(cell.T,
#                                  coords.T).T

#     if wrap:
#         for i, periodic in enumerate(pbc):
#             if periodic:
#                 # Yes, we need to do it twice.
#                 # See the scaled_positions.py test.
#                 fractional[:, i] %= 1.0
#                 fractional[:, i] %= 1.0
#     return fractional


# @curry
# def get_real_positions(coords, cell):
#     """Get position in real space coordinates"""
#     return np.dot(cell.T, coords.T).T


# def sphere(r=10):
#     """
#     args: radius of the sphere

#     returns: A 3D cubic matric of dim (2*r+1)^1
#     """
#     return pipe(2*r+1,
#                 lambda x: np.mgrid[:x,:x,:x],
#                 lambda xx: (xx[0]-r)**2 + (xx[1]-r)**2+(xx[2]-r)**2,
#                 lambda x: (x<r*r)*1)

# @curry
# def get_kdTree(coords, cell_dim, cutoff):
#     import MDAnalysis
    
#     tree = MDAnalysis.lib.pkdtree.PeriodicKDTree(box=cell_dim.astype(np.float32))
#     tree.set_coords(coords.astype(np.float32), 
#                     cutoff=np.float32(cutoff))
#     return tree


# def get_realStats(coords_all, coords_sub, indexes, r_stat, cell, pbc):
    
#     scaled_coords = get_scaled_positions(cell=cell, pbc=pbc, wrap=True)
#     real_coords = get_real_positions(cell=cell)
#     rescale = compose(real_coords, scaled_coords)
    
#     return pipe(indexes, 
#                 lambda indxs: rescale(coords_all[indxs[:,1]] - coords_sub[indxs[:,0]] + cell.diagonal()/2), 
#                 lambda crds: crds - cell.diagonal()/2)

# get_voxel_ids = lambda coords, len_pixel: (np.round(coords * len_pixel)).astype(int)
# get_dists = lambda c: np.sqrt(np.sum(c**2, axis=1))


# def get_rdf(coords_stat, r_stat, len_pixel):
    
#     nbins = np.round(r_stat*len_pixel).astype(int)+1
#     bins = np.linspace(0.0, r_stat+2, num=nbins)
    
#     dists = get_dists(coords_stat)
#     rdf, bin_edges = np.histogram(dists, bins)

#     bin_centers = (bin_edges[1:] + bin_edges[:-1])/2
    
#     rdf = rdf / rdf[0]
#     rdf[0] = 0.
#     vols = 4 / 3 * np.pi * (bin_edges[1:]**3 - bin_edges[:-1]**3)
#     pdf = rdf/vols
    
#     return rdf, pdf, bin_centers


# def get_2ptStat(coords_stat, n_coords, r_stat, len_pixel):
    
#     coords_indx = get_voxel_ids(coords_stat+r_stat, len_pixel=len_pixel)
#     shape = np.asarray([int(r_stat * 2 * len_pixel + 1)] * 3)
#     box = box_count(np.zeros(shape), 
#                      coords_indx, 
#                      len(coords_indx), shape)
    
#     return box / box.max()


# @numba.njit(parallel=True)
# def box_count(box, indexes, N, shape):
#     sx, sy, sz = shape
#     for i in range(N):
#         cx, cy, cz = indexes[i]
#         if (cx < sx) and (cx >= 0):
#             if (cy < sy) and (cy >= 0):
#                 if (cz < sz) and (cz >= 0):
#                     box[cx, cy, cz] += 1
#     return box

In [11]:
@curry
def generate(run, cutoff=15.0, r_stat=7.5, folder_out=None):
    atoms = pipe("data/structs/dump.arc.{}_atom.pkl".format(run), 
             lambda f: utils.load_file(f), 
             lambda a: get_atoms(a))

    idx = np.where(atoms.get_tags() != 1)[0]
    print("run: {}, total atoms: {}, disturbed atoms: {}".format(run, len(atoms), len(idx)))

    if len(idx) == 0:
        np.random.seed(81396)
        idx = np.random.randint(low=0, high=len(atoms), size=10000)


    coords_all = atoms.get_positions()
    coords_sub = coords_all[idx]
    
    tree = get_kdTree(atoms.get_positions(), 
                  cell_dim=atoms.get_cell_lengths_and_angles(), 
                 cutoff=cutoff 
                 )
    
    indexes = tree.search_tree(coords_sub, radius=r_stat)
    

    f_stat = "{}/dump.arc.{}_stat.pkl".format(folder_out, run)
    obj = {}
    obj["indexes"]=indexes
    obj["tree"]=tree
    utils.save_file(f_stat, obj=obj)

In [10]:
# import multiprocessing as mp
# n_proc = 8

# cutoff = 16.0
# r_stat = 15.0
# len_pixel = 10.0 


# folder = "data/stat_indxs_cutoff_{}".format(int(r_stat*10))
# print(folder)
# if not os.path.exists(folder):
#     os.mkdir(folder)

# with mp.Pool(n_proc) as p:
#     print(p.map(generate(cutoff=cutoff, 
#                          r_stat=r_stat, 
#                          folder_out=folder), runs))

data/stat_indxs_cutoff_150
run: 9000, total atoms: 4000000, disturbed atoms: 266384
run: 6000, total atoms: 4000000, disturbed atoms: 272163
run: 0, total atoms: 4000000, disturbed atoms: 0
run: 4500, total atoms: 4000000, disturbed atoms: 242254
run: 3000, total atoms: 4000000, disturbed atoms: 207399
run: 10500, total atoms: 4000000, disturbed atoms: 263228
run: 7500, total atoms: 4000000, disturbed atoms: 254992
run: 1500, total atoms: 4000000, disturbed atoms: 171079
run: 500, total atoms: 4000000, disturbed atoms: 87589
run: 1000, total atoms: 4000000, disturbed atoms: 132505
run: 2000, total atoms: 4000000, disturbed atoms: 182610
run: 3500, total atoms: 4000000, disturbed atoms: 206487
run: 12000, total atoms: 4000000, disturbed atoms: 270915
run: 5000, total atoms: 4000000, disturbed atoms: 244316
run: 9500, total atoms: 4000000, disturbed atoms: 261720
run: 8000, total atoms: 4000000, disturbed atoms: 289464
run: 11000, total atoms: 4000000, disturbed atoms: 263316
run: 6500, 